# News and Podcast Aggregator Agent using PraisonAIAgents

This notebook demonstrates a News and Podcast Aggregator Agent built with `praisonaiagents`.  
It summarizes news articles and generates podcast-style discussion points.  
The notebook is CI-friendly: it uses dummy data if API keys are not set, so all tests/checks will pass.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/SocialMedia_Content_Agents/News_And_Podcast_Aggregator_Agent.ipynb)


# Dependencies

In [ ]:
!pip install praisonaiagents pydantic

# Imports and Setup

In [3]:
import os

# Set up the OpenAI API key for praisonaiagents
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "sk-..")

def is_valid_key(key, prefix):
    return key and key != f"{prefix}-.." and key.startswith(prefix)

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Tools

In [4]:
from datetime import datetime
from typing import List

def format_search_results_for_podcast(search_results: List[dict]) -> tuple[str, List[str]]:
    """
    Custom tool to format search results for podcast script generation.
    """
    created_at = datetime.now().strftime("%B %d, %Y at %I:%M %p")
    structured_content = []
    structured_content.append(f"PODCAST CREATION: {created_at}\n")
    sources = []
    for idx, search_result in enumerate(search_results):
        try:
            if search_result.get("confirmed", False):
                sources.append(search_result["url"])
                structured_content.append(
                    f"""
                    SOURCE {idx + 1}:
                    Title: {search_result['title']}
                    URL: {search_result['url']}
                    Content: {search_result.get('full_text') or search_result.get('description', '')}
                    ---END OF SOURCE {idx + 1}---
                    """.strip()
                )
        except Exception as e:
            print(f"Error processing search result: {e}")
    content_texts = "\n\n".join(structured_content)
    return content_texts, sources

# Pydantic Models

In [5]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Dialog(BaseModel):
    speaker: str = Field(..., description="The speaker name (SHOULD BE 'ALEX' OR 'MORGAN')")
    text: str = Field(..., description="The spoken text content for this speaker based on the requested language, default is English")

class Section(BaseModel):
    type: str = Field(..., description="The section type (intro, headlines, article, outro)")
    title: Optional[str] = Field(None, description="Optional title for the section (required for article type)")
    dialog: List[Dialog] = Field(..., description="List of dialog exchanges between speakers")

class PodcastScript(BaseModel):
    title: str = Field(..., description="The podcast episode title with date")
    sections: List[Section] = Field(..., description="List of podcast sections (intro, headlines, articles, outro)")

# Agent Instructions

In [6]:
from textwrap import dedent

PODCAST_AGENT_DESCRIPTION = "You are a helpful assistant that can generate engaging podcast scripts for the given sources."
PODCAST_AGENT_INSTRUCTIONS = dedent("""
    You are a helpful assistant that can generate engaging podcast scripts for the given source content and query.
    For given content, create an engaging podcast script that should be at least 15 minutes worth of content and you are allowed to enhance the script beyond given sources if you know something additional info will be interesting to the discussion or not enough content is available.
    You use the provided sources to ground your podcast script generation process. Keep it engaging and interesting.
    IMPORTANT: Generate the entire script in the provided language.
    CONTENT GUIDELINES:
    - Provide insightful analysis that helps the audience understand the significance
    - Include discussions on potential implications and broader context of each story
    - Explain complex concepts in an accessible but thorough manner
    - Make connections between current and relevant historical developments when applicable
    - Provide comparisons and contrasts with similar stories or trends when relevant
    PERSONALITY NOTES:
    - Alex is more analytical and fact-focused
    - Morgan is more focused on human impact, social context, and practical applications
    - Include natural, conversational banter and smooth transitions between topics
    - Each article discussion should go beyond the basic summary to provide valuable insights
    - Maintain a conversational but informed tone that would appeal to a general audience
    IMPORTANT:
        - MAKE SURE PODCAST SCRIPTS ARE AT LEAST 15 MINUTES LONG WHICH MEANS YOU NEED TO HAVE DETAILED DISCUSSIONS BUT KEEP IT INTERESTING AND ENGAGING.
""")

# Dummy Data and Main Logic

In [7]:
from praisonaiagents import Agent

# Dummy search results for CI/public use
dummy_search_results = [
    {
        "confirmed": True,
        "title": "AI Revolutionizes Healthcare",
        "url": "https://news.example.com/ai-healthcare",
        "full_text": "Artificial intelligence is transforming the healthcare industry by..."
    },
    {
        "confirmed": True,
        "title": "Climate Change and Global Policy",
        "url": "https://news.example.com/climate-policy",
        "full_text": "World leaders are meeting to discuss new policies for climate change..."
    }
]

query = "Latest technology and world news"
language_name = "English"

content_texts, sources = format_search_results_for_podcast(dummy_search_results)

if not is_valid_key(OPENAI_API_KEY, "sk"):
    print("API key not set or is a placeholder. Using dummy podcast script for CI/testing.")
    print("=== Podcast Script ===")
    print(f"Podcast Title: Tech & World News Roundup ({datetime.now().strftime('%B %d, %Y')})")
    for idx, article in enumerate(dummy_search_results):
        print(f"\nSection {idx+1}: {article['title']}")
        print(f"Summary: {article['full_text'][:80]}...")
else:
    agent = Agent(
        name="Podcast Script Agent",
        instructions=PODCAST_AGENT_INSTRUCTIONS,
        api_key=OPENAI_API_KEY
    )
    prompt = f"query: {query}\nlanguage_name: {language_name}\ncontent_texts: {content_texts}\nIMPORTANT: texts should be in {language_name} language."
    podcast_script = agent.start(prompt)
    print("=== Podcast Script ===")
    print(podcast_script)

╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Podcast Script Agent                                                                                 │
│  Role: Assistant                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Response generated in 23.5s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ query: Latest technology and world news language_name: English content_texts: PODCAST CREATION: July 09, 2025   │
│ at 10:03 AM                                                                                                     │
│                                                                                                                 │
│ SOURCE 1: Title: AI Revolutionizes Healthcare URL: https://news.example.com/ai-healthcare Content: Artificial   │
│ intelligence is transforming the healthcare industry by... ---END OF SOURCE 1---                                │
│                                                                                                                 │
│ SOURCE 2: Title: Climate Change and Global Policy URL: https://news.example.com/climate-policy Content: World   │
│ leaders are meeting to discuss new policies for climate change... ---END OF SOURCE 2--- IMPORTANT: texts should │
│ be in English language.                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ [INTRO MUSIC FADES IN]                                                                                          │
│                                                                                                                 │
│ Alex: Hello and welcome to another episode of "The World Today," where we dive into the latest in technology    │
│ and world news. I'm Alex, your go-to for all things analytical and fact-focused.                                │
│                                                                                                                 │
│ Morgan: And I'm Morgan, here to explore the human impact and social context of today's stories. We've got a     │
│ fascinating lineup for you today, so let's dive right in!                                                       │
│                                                                                                                 │
│ Alex: First up, we're talking about a topic that's been making waves across the healthcare industry: the role   │
│ of artificial intelligence. According to a recent article from ]8;id=890061;https://news.example.com/ai-healthcare\News Example]8;;\, AI is revolutionizing healthcare   │
│ in ways we could only dream of a decade ago.                                                                    │
│                                                                                                                 │
│ Morgan: That's right, Alex. AI is not just a buzzword anymore; it's actively changing how we approach           │
│ healthcare. From diagnostics to treatment plans, AI is helping doctors make more accurate decisions faster. But │
│ what does this mean for patients and healthcare professionals?                                                  │
│                                                                                                                 │
│ Alex: Well, Morgan, AI's ability to analyze vast amounts of data quickly means that doctors can diagnose        │
│ diseases earlier and more accurately. For instance, AI algorithms can now detect patterns in medical imaging    │
│ that might be too subtle for the human eye. This can lead to earlier interventions and better patient outcomes. │
│                                                                                                                 │
│ Morgan: And let's not forget about the human side of things. For patients, this means less waiting time for     │
│ results and potentially more personalized treatment plans. But there's also a concern about the human touch in  │
│ healthcare. How do we balance the efficiency of AI with the empathy and understanding that only a human can     │
│ provide?                                                                                                        │
│                                                                                                                 │
│ Alex: That's a great point. The integration of AI in healthcare does raise ethical questions. For example, who  │
│ is responsible if an AI system makes a mistake? And how do we ensure that these systems are free from bias?     │
│                                                                                                                 │
│ Morgan: Exactly. There's a lot to consider, but the potential benefits are enormous. Imagine a world where      │
│ healthcare is not only more efficient but also more accessible to people in remote areas thanks to AI-driven    │
│ telemedicine.                                                                                                   │
│                                                                                                                 │
│ Alex: It's a fascinating time for healthcare, no doubt. But let's shift gears a bit and talk about another   

=== Podcast Script ===
[INTRO MUSIC FADES IN]

**Alex**: Hello and welcome to another episode of "The World Today," where we dive into the latest in technology and world news. I'm Alex, your go-to for all things analytical and fact-focused.

**Morgan**: And I'm Morgan, here to explore the human impact and social context of today's stories. We've got a fascinating lineup for you today, so let's dive right in!

**Alex**: First up, we're talking about a topic that's been making waves across the healthcare industry: the role of artificial intelligence. According to a recent article from [News Example](https://news.example.com/ai-healthcare), AI is revolutionizing healthcare in ways we could only dream of a decade ago.

**Morgan**: That's right, Alex. AI is not just a buzzword anymore; it's actively changing how we approach healthcare. From diagnostics to treatment plans, AI is helping doctors make more accurate decisions faster. But what does this mean for patients and healthcare professio